In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

from config import league_info, companies_training

In [2]:
# To concatenate with league name for csv file to be read
csv_concat = "_S2018-2019.csv"

In [3]:
# .......... FUNCTION TO BUILD CLASSIFIER FOR NEURAL NETWORK MODEL .......... #
def build_classifier(optimizer):
    classifier = Sequential()

    # Hidden layer
    classifier.add(Dense(units=3,     # Unit determination see below
                         activation="relu",
                         input_dim=len(f)                         
                        ))
    # Output layer
    classifier.add(Dense(units=3,     # Output
                         activation="softmax"
                        ))
    # Compile the neural work
    classifier.compile(
        optimizer=optimizer,
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    
    return classifier
# .......... FUNCTION ENDS .......... #

'''
NUMBER OF HIDDEN NEURONS IN NEURAL NETWORK MODEL

According to the ref:
https://stats.stackexchange.com/questions/181/
    how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw
1. Layer of hidden nodes is set to 1
2. Number of input neurons is the length of features (4)
3. Number of output neurons is 3
4. Number of hidden neurons (hn) is supposed to be selected between 3 (output) and 4 (input), exclusive

In this study, the unit is set as 3
'''

'\nNUMBER OF HIDDEN NEURONS IN NEURAL NETWORK MODEL\n\nAccording to the ref:\nhttps://stats.stackexchange.com/questions/181/\n    how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw\n1. Layer of hidden nodes is set to 1\n2. Number of input neurons is the length of features (4)\n3. Number of output neurons is 3\n4. Number of hidden neurons (hn) is supposed to be selected between 3 (output) and 4 (input), exclusive\n\nIn this study, the unit is set as 3\n'

In [4]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.ensemble import RandomForestClassifier    # Random Forest (RF) model
from sklearn.svm import SVC    # Support Vector Machine (SVM) model
from sklearn.neighbors import KNeighborsClassifier    # K Nearest Neighbors (KNN) model
from tensorflow.keras.models import Sequential   # Neural Network model
from tensorflow.keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

from numpy.random import seed
import pickle

# DataFrame to store data distribution for viz
distr_df = pd.DataFrame(columns=["league", "win_odds_s", "win_odds_k", "draw_odds_s", "draw_odds_k",
                                "lose_odds_s", "lose_odds_k", "odds_delta_time_s", "odds_delta_time_k"])

# Dict to store established model info. before comparison
model_estb = {}

# DataFrame to hold socre info for ML models
score_df = pd.DataFrame(columns=[
    "param", "training_score", "testing_score", 
    "precision_draw", "precision_lose", "precision_win",
    "recall_draw", "recall_lose", "recall_win",
    "f1_draw", "f1_lose", "f1_win", "n_estimators"
])

# Loop through leagues
for league in list(league_info.keys()):
    
    # Read csv data
    data = pd.read_csv(f'../data/{league}{csv_concat}')                       
    # Convert data type of "odds_delta_time" from to float to kill "DataConversionWarning" for "MinMaxScaler()"
    data["odds_delta_time"] = data["odds_delta_time"].astype("float")
                       
    # Index of "distr_df" used to append new data for each league
    distr_idx = len(distr_df)
    # Append league info. to "distr_idx"
    distr_df.loc[distr_idx, "league"] = league

    # Loop through companies
    for company in list(companies_training.values()):
        
        # Dict to store data divided by company
        data_company = {}
        # Divide data by company               
        data_company["ori"] = data.loc[data.company == company, :]
        # Generate altenative company-divided data with odds time under the limit determined in Tableau
        data_company["alt"] = data_company["ori"].loc[data_company["ori"].odds_delta_time <=\
            league_info[league]["odds_limit"], :]
                              
        # .......... Data Distribution .......... #
        # Columns to be considered as features for machine learning models
        feature_f4 = ["win_odds", "draw_odds", "lose_odds", "odds_delta_time"]        
        # "According to corr analysis in Tableau, "lose_odds" might be redundant to serve as feature             
        feature_f3 = ["win_odds", "draw_odds", "odds_delta_time"]
        
        # Determine data distribution for features to be used to generate machine learning (ML) models
        for feature in feature_f4:
            
            skewness = stats.describe(data_company["ori"][feature])[4]
            kurtosis = stats.describe(data_company["ori"][feature])[5]
            # Save skewness and kurtosis to "distr_df"           
            distr_df.loc[distr_idx, f"{feature}_s"] = skewness
            distr_df.loc[distr_idx, f"{feature}_k"] = kurtosis
                       
        ########################################################################################
        # BASED ON SKEWNESS AND KURTOSIS, THERE IS SELDOM FEATUER THAT IS NORMALLY DISTRIBUTED.#
        # WE WILL APPLY RANDOM FOREST, SVM, KNN, AND NEURAL NETWORK MODELS THEREAFTER          #
        ########################################################################################
                       
        # .......... Prep. for ML .......... #
        # Column to be used as label(target) of ML model
        for tgt in ["ori", "alt"]:
            
            # Target
            target = data_company[tgt]["result"]
            # Target encoder
            t_encoder = LabelEncoder().fit(target.values)
            # Transform categorical labels to numerical
            target_encd = t_encoder.transform(target.values)
            # One-hot encoding labels
            target_encd_categ = to_categorical(target_encd)
        
            # Columns to be used as fetures of ML model
            for f in [feature_f4, feature_f3]:
                features = data_company[tgt][f]
                       
                # Set up train and test data for all machine learning models to be built in this project
                X_train, X_test, y_train, y_test =\
                    train_test_split(features, target_encd_categ, random_state=8)
                       
                # MinMaxScaler is preferred over StandardScaler for scaling distance features
                X_scaler = MinMaxScaler().fit(X_train)
                # Scale "X-train" and "X-test"
                X_train_scaled = X_scaler.transform(X_train)
                X_test_scaled = X_scaler.transform(X_test)                               

                seed(8)

                
                # ML models to be trained below
                models = ['rf', 'svm', 'knn', 'nn']
                
                # Dict to hold all info. with regard to ML models
                model_info = {}
                
                # Classifiers
                model_info["rf_classifier"] = RandomForestClassifier(random_state=8)
                model_info["svm_classifier"] = SVC(random_state=8)
                model_info["knn_classifier"] = KNeighborsClassifier()                       

                # Wrap tensorflow model so it can be used by scikit-learn
                model_info["nn_classifier"] = KerasClassifier(build_fn=build_classifier, verbose=0)  

                # ************************************************************* #
                # Use default setting of classifiers for the initial screening  #
                # For KNN, set k as even numbers that are less than 20          #
                # ************************************************************* #
                       
                # Default parameter for each classifier
                model_info["rf_param_grid"] = {
                    # The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22
                    'n_estimators': [100]
                }

                model_info["svm_param_grid"] = {
                    # The default value of gamma will change from 'auto' to 'scale' in version 0.22.
                    'gamma': ["scale"]
                }

                model_info["knn_param_grid"] = {
                    'n_neighbors': list(range(1, 20, 2))
                }


                model_info["nn_param_grid"] = {
                    'optimizer': ["adam"]
                }                                              
                       
                # Loop through models             
                for model in models:

                    # Print current iterating model
                    print("*" * 65)
                    print(f'Current iterating: {league}|{company}|{tgt}|{features.shape[1]}|{model.upper()}')

                    # Create GridSearch estimator
                    model_info[f"{model}_estimator"] = GridSearchCV(
                        model_info[f"{model}_classifier"], 
                        model_info[f"{model}_param_grid"],
                        # Default value of cv will change from 3 to 5 in version 0.22.
                        # Specify it to kill "FutureWarning"
                        cv=5
                    )

                    # SVM model can not take One-Hot-encoding label
                    if model == "svm":
                        # Tempprary y_train and y_test used for model fitting
                        y_train_im = np.argmax(y_train, axis=-1) 
                        y_test_im = np.argmax(y_test, axis=-1) 
                    else:
                        y_train_im = y_train 
                        y_test_im = y_test    

                    # Fit model using grid search estimator
                    if model == "nn":
                        # Shuffling is recommended for neural network model
                        # Set "epochs" to 1000
                        model_info[f"{model}_estimator"].\
                            fit(X_train_scaled, y_train_im, epochs=100, shuffle=True)
                    else:
                        model_info[f"{model}_estimator"].fit(X_train_scaled, y_train_im)

                    # Inverse labels in "y_test"
                    # https://stackoverflow.com/questions/54143458/
                    #    convert-categorical-data-back-to-numbers-using-keras-utils-to-categorical
                    y_test_invr = t_encoder.inverse_transform(np.argmax(y_test, axis=-1))

                    # Predict "y_pred"
                    y_pred = model_info[f"{model}_estimator"].predict(X_test_scaled)

                    # Inverse labels in "y_pred"
                    # Note that Sequential predict always returns the same datatype of "y_train_im"
                    # "predict" for KerasClassifier("nn") returns CLASS prediction for given test data 
                    # (like "predict_classes for Sequential")
                    if (model == "svm") or (model == "nn"):
                        y_pred_invr = t_encoder.inverse_transform(y_pred)
                    else:
                        y_pred_invr = t_encoder.inverse_transform(np.argmax(y_pred, axis=-1))
                        
                    # .......... MODEL EVALUATION .......... #    
                    # Length of current score_df to be used as the index of rows new data is written to
                    score_idx = len(score_df)
                    
                    # Iterated param for score_df
                    score_df.loc[score_idx, "param"] =\
                        f"{league}_{company}_{tgt}_{features.shape[1]}_{model}"            

                    # Scores
                    # Note that KerasClassifier also has "score" (mean accuracy)
                    score_df.loc[score_idx, "training_score"] =\
                        model_info[f"{model}_estimator"].score(X_train_scaled, y_train_im)
                    score_df.loc[score_idx, "testing_score"] =\
                        model_info[f"{model}_estimator"].score(X_test_scaled, y_test_im)
                    
                    # Classification report
                    classif = classification_report(y_test_invr, y_pred_invr).split(" ")
                    while "" in classif:
                        classif.remove("")                                                                              
                    # Store "precision", "recall", and "f1-score" info for "DRAW", "LOSE", and "WIN"
                    score_df.loc[score_idx, "precision_draw"] = float(classif[5])
                    score_df.loc[score_idx, "recall_draw"] = float(classif[6])
                    score_df.loc[score_idx, "f1_draw"] = float(classif[7])
                    score_df.loc[score_idx, "precision_lose"] = float(classif[10])
                    score_df.loc[score_idx, "recall_lose"] = float(classif[11])
                    score_df.loc[score_idx, "f1_lose"] = float(classif[12])                          
                    score_df.loc[score_idx, "precision_win"] = float(classif[15])
                    score_df.loc[score_idx, "recall_win"] = float(classif[16])
                    score_df.loc[score_idx, "f1_win"] = float(classif[17])                                              
                       
                    # "n_estimators" 
                    if model != "knn":
                         score_df.loc[score_idx, "n_estimators"] = "N/A"
                    else:
                        score_df.loc[score_idx, "n_estimators"] =\
                            str(model_info[f"{model}_estimator"]).split("n_neighbors=")[1].split(",")[0]
                       
                    # Print complete information
                    print(f'{league}|{company}|{tgt}|{features.shape[1]}|{model.upper()} DONE!!!\n')                       



# Save score data as csv file
score_df.to_csv(f"../data/data_for_viz/ml_scores.csv", index=False, header=True)
                                        
# Save feature distribution data as csv file
distr_df.to_csv(f"../data/data_for_viz/data_distribution.csv", index=False, header=True)

Using TensorFlow backend.


*****************************************************************
Current iterating: PremierLeague|Bet365|ori|4|RF
PremierLeague_Bet365_ori_4_rf
PremierLeague|Bet365|ori|4|RF DONE!!!

*****************************************************************
Current iterating: PremierLeague|Bet365|ori|4|SVM
PremierLeague_Bet365_ori_4_svm
PremierLeague|Bet365|ori|4|SVM DONE!!!

*****************************************************************
Current iterating: PremierLeague|Bet365|ori|4|KNN
PremierLeague_Bet365_ori_4_knn


W0804 04:19:30.088770 13864 deprecation.py:506] From C:\Users\kouda\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


PremierLeague|Bet365|ori|4|KNN DONE!!!

*****************************************************************
Current iterating: PremierLeague|Bet365|ori|4|NN
PremierLeague_Bet365_ori_4_nn


C:\Users\kouda\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


...

MLS|12Bet|ori|3|NN DONE!!!

*****************************************************************
Current iterating: MLS|12Bet|alt|4|RF
MLS_12Bet_alt_4_rf
MLS|12Bet|alt|4|RF DONE!!!

*****************************************************************
Current iterating: MLS|12Bet|alt|4|SVM
MLS_12Bet_alt_4_svm
MLS|12Bet|alt|4|SVM DONE!!!

*****************************************************************
Current iterating: MLS|12Bet|alt|4|KNN
MLS_12Bet_alt_4_knn
MLS|12Bet|alt|4|KNN DONE!!!

*****************************************************************
Current iterating: MLS|12Bet|alt|4|NN
MLS_12Bet_alt_4_nn


C:\Users\kouda\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


MLS|12Bet|alt|4|NN DONE!!!

*****************************************************************
Current iterating: MLS|12Bet|alt|3|RF
MLS_12Bet_alt_3_rf
MLS|12Bet|alt|3|RF DONE!!!

*****************************************************************
Current iterating: MLS|12Bet|alt|3|SVM
MLS_12Bet_alt_3_svm
MLS|12Bet|alt|3|SVM DONE!!!

*****************************************************************
Current iterating: MLS|12Bet|alt|3|KNN
MLS_12Bet_alt_3_knn
MLS|12Bet|alt|3|KNN DONE!!!

*****************************************************************
Current iterating: MLS|12Bet|alt|3|NN
MLS_12Bet_alt_3_nn


C:\Users\kouda\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


MLS|12Bet|alt|3|NN DONE!!!



##### Analysis from Tableau
<p><strong>Top scored:</strong></p>
<img src="../static/image/ipynb/top_scored.png" alt="top scored">
<p><strong>Testing score:</strong></p>
<img src="../static/image/ipynb/testing_score.png" alt="testing score">

##### After viz and analysed in Tableau, the following models were selected for further fine tuning:
    PremierLeague_12Bet_ori_f4_knn/rf
    LaLiga_12Bet_ori_f4_knn/rf
    Bundesliga_12Bet_alt_f4_knn/rf
    Serie-A_12Bet_alt_f4_knn/rf
    Ligue1_12Bet_alt_f4_knn/rf
    MLS_12Bet_alt_f4_knn/rf
    PremierLeague_WilliamHill_ori_f4_rf (Alternative model from another betting company other than 12Bet)
    
    From the results:
    1. 12Bet is the best betting company we might want to retrieve odds-movement data from
    2. All selected models were either RF or KNN with n_neighbors=5
    3. Removal of odds that were too "old" facilitates for two thirds of leagues in building up ML models
    4. Despite a high correlation with "draw_odds" (and "win_odds" to some point), retaining "lose_odds" as feature is beneficial in building up ML models
    5. As indicated in "UndefinedMetricWarning", scores for "DRAW" data might not be reliable due to relatively small amounts of "DRAW" records for the match as compared to "WIN" and "LOSE" (imbalance in labels)

In [5]:
# List to hold selected ML model params
model_sel = [
    "PremierLeague_12Bet_ori_f4_knn",
    "PremierLeague_12Bet_ori_f4_rf",
    "LaLiga_12Bet_ori_f4_knn",
    "LaLiga_12Bet_ori_f4_rf",    
    "Bundesliga_12Bet_alt_f4_knn",
    "Bundesliga_12Bet_alt_f4_rf",    
    "Serie-A_12Bet_alt_f4_knn",
    "Serie-A_12Bet_alt_f4_rf",    
    "Ligue1_12Bet_alt_f4_knn",
    "Ligue1_12Bet_alt_f4_rf",    
    "MLS_12Bet_alt_f4_knn",
    "MLS_12Bet_alt_f4_rf",    
    "PremierLeague_WilliamHill_ori_f4_rf"
]

In [11]:
# KNN does not support model updating, re-training the same data for fine tuning
# Dict to store established model info. before comparison
model_estb = {}

# DataFrame to hold socre info for ML models
score_df = pd.DataFrame(columns=[
    "param", "training_score", "testing_score", 
    "precision_draw", "precision_lose", "precision_win",
    "recall_draw", "recall_lose", "recall_win",
    "f1_draw", "f1_lose", "f1_win"
])

# Loop through names in "model_sel"
for name in model_sel:
    
    # Retrieve params from iterated name
    [league, company, tgt, f, model] = name.split("_")

    # KNN Classifier to be fine tuned
    model_estb["classifier_rf"] = RandomForestClassifier(random_state=8)
    model_estb["classifier_knn"] = KNeighborsClassifier(n_neighbors=5)
    

    # Set up tuning parameters for each classifier
    model_estb["param_grid_rf"] = {
        'n_estimators': [100, 500],
        'criterion': ["gini", "entropy"],
        'max_features': ["auto", "sqrt", "log2"]
    }
    model_estb["param_grid_knn"] = {
        'weights': ["uniform", "distance"],
        'algorithm': ["ball_tree", "kd_tree", "brute", "auto"],
        'leaf_size': [30, 300]
    }    
    
    # Read csv data
    data = pd.read_csv(f'../data/{league}{csv_concat}')    
    # Retrieve data from company indicated in model
    cdata = data.loc[data.company == company, :]
    
    # Determine data (tdata) used for fine tuning our machine models
    if (league == "PremierLeague") or (league == "LaLiga"):        
        tdata = cdata        
    else:
        tdata = cdata.loc[cdata.odds_delta_time <= league_info[league]["odds_limit"], :]      
    
    # Column to be used as label(target) for tuning
    ttarget = tdata.result   

    # "result" for every league has the save unique value of "W", "D", and "L"
    # Removing a tiny small portion of "result" won't affect the variety of "result"
    # Therefore, "t_encoder" from the last iteration will be saved for every model prediction 
    tt_encoder = LabelEncoder().fit(ttarget.values)
    # Transform categorical labels to numerical
    ttarget_encd = tt_encoder.transform(ttarget.values)       
    # One-hot encoding labels
    ttarget_encd_categ = to_categorical(ttarget_encd)    
    
    # Columns to be used as fetures of tuning
    tfeatures = tdata[feature_f4]
    
    # Set up train and test data for all machine learning models to be built in this project
    X_train, X_test, y_train, y_test = train_test_split(tfeatures, ttarget_encd_categ, random_state=8)

    # MinMaxScaler is preferred over StandardScaler for scaling distance features
    model_estb[f"X_scaler_{league}"] = MinMaxScaler().fit(X_train)

    X_train_scaled = model_estb[f"X_scaler_{league}"].transform(X_train)
    X_test_scaled = model_estb[f"X_scaler_{league}"].transform(X_test)   
    
    # Print current iterating model
    print("*" * 65)
    print(f'Current iterating: {league}|{company}|{model.upper()}\n')    
    
    # Create GridSearch estimator
    model_estb[f"estimator_{model}"] = GridSearchCV(
        model_estb[f"classifier_{model}"], 
        model_estb[f"param_grid_{model}"],
        # Default value of cv will change from 3 to 5 in version 0.22. Specify it to kill "FutureWarning"
        cv=5
    )    
    
    # Fit model using grid search estimator
    model_estb[f"estimator_{model}"].fit(X_train_scaled, y_train)
    
    # Inverse labels in "y_test"
    y_test_invr = tt_encoder.inverse_transform(np.argmax(y_test, axis=-1))
    
    # Predict "y_pred"
    y_pred = model_estb[f"estimator_{model}"].predict(X_test_scaled)
    # Inverse labels in "y_pred"
    y_pred_invr = tt_encoder.inverse_transform(np.argmax(y_pred, axis=-1))     
    
    # .......... MODEL EVALUATION .......... #
    # Length of current score_df to be used as the index of rows new data is written to
    score_idx = len(score_df)

    # Iterated param for score_df
    score_df.loc[score_idx, "param"] = f"{league}_{company}_{model}"            

    # Scores
    # Note that KerasClassifier also has "score" (mean accuracy)
    score_df.loc[score_idx, "training_score"] =\
        model_estb[f"estimator_{model}"].score(X_train_scaled, y_train)
    score_df.loc[score_idx, "testing_score"] =\
        model_estb[f"estimator_{model}"].score(X_test_scaled, y_test)

    # Classification report
    classif = classification_report(y_test_invr, y_pred_invr).split(" ")
    while "" in classif:
        classif.remove("")                                                                              
    # Store "precision", "recall", and "f1-score" info for "DRAW", "LOSE", and "WIN"
    score_df.loc[score_idx, "precision_draw"] = float(classif[5])
    score_df.loc[score_idx, "recall_draw"] = float(classif[6])
    score_df.loc[score_idx, "f1_draw"] = float(classif[7])
    score_df.loc[score_idx, "precision_lose"] = float(classif[10])
    score_df.loc[score_idx, "recall_lose"] = float(classif[11])
    score_df.loc[score_idx, "f1_lose"] = float(classif[12])                          
    score_df.loc[score_idx, "precision_win"] = float(classif[15])
    score_df.loc[score_idx, "recall_win"] = float(classif[16])
    score_df.loc[score_idx, "f1_win"] = float(classif[17])
    
    # Print the comparison between "test" and "pred" for every model
    print(f'{score_df.loc[score_idx, "param"]}_test: {y_test_invr}')
    print(f'{score_df.loc[score_idx, "param"]}_pred: {y_pred_invr}\n')         
    
    # Save X-scaler
    x_scalerfile = f'../models/X-scaler_{league}.sav'
    pickle.dump(model_estb[f"X_scaler_{league}"], open(x_scalerfile, 'wb'))
    
    # Save model as .pkl file (documentation)
    with open(f"../models/{league}_{company}_{model}.pkl", 'wb') as fid:
        pickle.dump(model_estb[f"estimator_{model}"], fid)

    # Print complete information
    print(f'{league}|{company}|{model.upper()} DONE!!!\n')                       

# Save "tt_encoder"
y_encoderfile = '../models/target-encoder.sav'
pickle.dump(tt_encoder, open(y_encoderfile, 'wb'))    
    
# Save score data as csv file
score_df.to_csv(f"../data/data_for_viz/ml_scores_final.csv", index=False, header=True)

*****************************************************************
Current iterating: PremierLeague|12Bet|KNN

PremierLeague_12Bet_knn_test: ['L' 'L' 'W' ... 'L' 'L' 'W']
PremierLeague_12Bet_knn_pred: ['W' 'L' 'W' ... 'L' 'D' 'L']

PremierLeague|12Bet|KNN DONE!!!

*****************************************************************
Current iterating: PremierLeague|12Bet|RF

PremierLeague_12Bet_rf_test: ['L' 'L' 'W' ... 'L' 'L' 'W']
PremierLeague_12Bet_rf_pred: ['W' 'L' 'W' ... 'L' 'D' 'L']

PremierLeague|12Bet|RF DONE!!!

*****************************************************************
Current iterating: LaLiga|12Bet|KNN

LaLiga_12Bet_knn_test: ['W' 'D' 'W' ... 'D' 'W' 'W']
LaLiga_12Bet_knn_pred: ['W' 'D' 'W' ... 'D' 'D' 'W']

LaLiga|12Bet|KNN DONE!!!

*****************************************************************
Current iterating: LaLiga|12Bet|RF

LaLiga_12Bet_rf_test: ['W' 'D' 'W' ... 'D' 'W' 'W']
LaLiga_12Bet_rf_pred: ['W' 'D' 'W' ... 'D' 'L' 'W']

LaLiga|12Bet|RF DONE!!!

********

##### Analysis from Tableau
<img src="../static/image/ipynb/model_comparison_final.png" alt="model comparison">

<p>As can be told from the above analysis, <strong>RF</strong> models seemed to perform a little bit better both in <i>testing</i> and in <i>precision</i> and <i>recall</i> scores over <strong>KNN</strong>. Ideally we should be using RF models for prediction. However, since the size of saved RF models way exceeded Github's maximum file uploading requirement, I will instead use KNN model for demo in html.</p>